In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from main2 import run_simulation

# Cell 1: Run simulation and save data to CSV
def save_simulation_to_csv(episodes=1000, agent_type="DQN", empathy_level="balanced", 
                           env_size=6, num_agents=3, steps_per_episode=500, seed=42,
                           output_dir="simulation_data"):
    """
    Run a simulation and save all results to CSV files.
    
    Args:
        episodes: Number of episodes to run
        agent_type: Type of agent ("DQN" or "QLearning")
        empathy_level: Social reward profile
        env_size: Environment grid size
        num_agents: Number of agents
        steps_per_episode: Steps per episode
        seed: Random seed for reproducibility
        output_dir: Directory to save CSV files
    
    Returns:
        Dictionary with paths to saved files
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate a timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_filename = f"{agent_type}_{empathy_level}_{episodes}ep_{timestamp}"
    
    # Run the simulation
    print(f"Running simulation with {episodes} episodes...")
    results = run_simulation(
        episodes=episodes,
        agent_type=agent_type,
        empathy_level=empathy_level,
        env_size=env_size,
        num_agents=num_agents,
        steps_per_episode=steps_per_episode,
        seed=seed
    )
    print("Simulation complete. Saving data...")
    
    # Extract data from results
    episode_rewards = results['episode_rewards']
    social_welfare = results['social_welfare']
    agent_meals = results['agent_meals']
    action_counts = results['action_counts']
    
    # Save configuration as CSV
    config_df = pd.DataFrame([{
        'agent_type': agent_type,
        'empathy_level': empathy_level,
        'episodes': episodes,
        'env_size': env_size,
        'num_agents': num_agents,
        'steps_per_episode': steps_per_episode,
        'seed': seed
    }])
    config_path = os.path.join(output_dir, f"{base_filename}_config.csv")
    config_df.to_csv(config_path, index=False)
    
    # Save episode rewards and social welfare
    episode_df = pd.DataFrame({
        'episode': list(range(episodes)),
        'reward': episode_rewards,
        'social_welfare': social_welfare
    })
    episode_path = os.path.join(output_dir, f"{base_filename}_episode_data.csv")
    episode_df.to_csv(episode_path, index=False)
    
    # Save agent meals
    meals_data = {}
    for i in range(num_agents):
        meals_data[f'agent_{i+1}_meals'] = agent_meals[i]
    
    meals_df = pd.DataFrame({
        'episode': list(range(episodes)),
        **meals_data
    })
    meals_path = os.path.join(output_dir, f"{base_filename}_agent_meals.csv")
    meals_df.to_csv(meals_path, index=False)
    
    # Save action distributions
    # First, create a list to hold all the action data
    action_data = []
    
    for episode in range(episodes):
        episode_actions = action_counts[episode]
        
        for agent_id, counts in episode_actions.items():
            # Calculate percentages
            total = sum(counts)
            percentages = [count/total*100 if total > 0 else 0 for count in counts]
            
            # Add row to data
            row = {
                'episode': episode,
                'agent_id': agent_id,
                'up': counts[0],
                'down': counts[1],
                'left': counts[2],
                'right': counts[3],
                'exploit': counts[4],
                'up_pct': percentages[0],
                'down_pct': percentages[1],
                'left_pct': percentages[2],
                'right_pct': percentages[3],
                'exploit_pct': percentages[4]
            }
            action_data.append(row)
    
    # Create DataFrame and save
    actions_df = pd.DataFrame(action_data)
    actions_path = os.path.join(output_dir, f"{base_filename}_action_data.csv")
    actions_df.to_csv(actions_path, index=False)
    
    # Save paths to files
    saved_files = {
        'config': config_path,
        'episode_data': episode_path,
        'agent_meals': meals_path,
        'action_data': actions_path
    }
    
    print(f"Data saved to {output_dir} directory.")
    print(f"Files created:")
    for key, path in saved_files.items():
        print(f"- {os.path.basename(path)}")
    
    return saved_files

# Example usage
# Run this cell to execute a simulation and save results to CSV
saved_files = save_simulation_to_csv(
    episodes=500,
    agent_type="DQN",
    empathy_level="balanced",
    env_size=6,
    num_agents=3,
    steps_per_episode=100,
    seed=42
)

# Cell 2: Load and preview saved data
def load_simulation_data(files):
    """Load and preview the saved simulation data."""
    data = {}
    
    for key, path in files.items():
        data[key] = pd.read_csv(path)
        print(f"\n{key.upper()} DATA PREVIEW:")
        print(data[key].head())
    
    return data

# Load and preview the data we just saved
simulation_data = load_simulation_data(saved_files)

# Cell 3: Run comparative simulations and save results
def run_comparative_simulations(episodes=500, seeds=None):
    """
    Run simulations with different agent types and empathy levels,
    saving all results to CSV.
    """
    if seeds is None:
        seeds = [42, 43, 44]
    
    agent_types = ["DQN", "QLearning"]
    empathy_levels = ["high_empathy", "balanced", "low_empathy"]
    
    all_files = []
    
    for agent_type in agent_types:
        for empathy in empathy_levels:
            for seed in seeds:
                print(f"\nRunning simulation: {agent_type}, {empathy}, seed={seed}")
                files = save_simulation_to_csv(
                    episodes=episodes,
                    agent_type=agent_type,
                    empathy_level=empathy,
                    seed=seed
                )
                all_files.append({
                    'agent_type': agent_type,
                    'empathy_level': empathy,
                    'seed': seed,
                    'files': files
                })
    
    return all_files

# Uncomment to run comparative simulations
# comparative_results = run_comparative_simulations(episodes=300)

ImportError: cannot import name 'run_simulation' from 'main2' (c:\Users\evann\OneDrive\Documents\GitHub\empathy_tragedy\2D\main2.py)